In [1]:
import oracledb as ora
import os
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
#관광 지출액 데이터
df = pd.read_excel("최종데이터.xlsx")

#GDP 데이터
gdp=pd.read_excel("GDP.xlsx")

#CPI 데이터
cpi=pd.read_excel("소비자 물가지수.xlsx")

#기대인플레이션율
expect_inflaction=pd.read_excel("기대 인플레이션율.xlsx")

#지역별 방문객
visitor=pd.read_excel("지역별 방문자 수.xlsx")

In [3]:
#항목별 컬럼명
new_columns = ['호텔', '콘도', '캠핑장/펜션', '기타숙박', '면세점', '관관기념품', '레저용품쇼핑', '대형쇼핑몰', '관광유원시설', '골프장','스키장', '기타레저', '문화서비스', '식음료']

#소비데이터 - 충청북도
chungbuk_df =df[df['시도구분'] == '충청북도']
chungbuk_df=chungbuk_df.transpose()
chungbuk_df.columns = new_columns
chungbuk_df=chungbuk_df.drop(chungbuk_df.index[0])
chungbuk_df=chungbuk_df.drop(chungbuk_df.index[0])

In [4]:
#gdp 행렬 변환
gdp=gdp.transpose()

#gdp unnammed 행 제거
gdp= gdp.drop(gdp.index[0])

#기대인플레이션율
expect_inflaction=expect_inflaction.transpose()

#기대인플레이션율 unnammed 행 제거
expect_inflaction= expect_inflaction.drop(expect_inflaction.index[0])

#CPI 행렬 변환
cpi=cpi.transpose()

#CPI unnammed 행 제거
cpi= cpi.drop(cpi.index[0])

In [5]:
#방문객 데이터 전처리
visitor=visitor.transpose()
visitor=visitor.drop(visitor.index[0])
new_columns = ['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']
visitor.columns = new_columns
visitor_gangwon= visitor[['강원도']]
visitor_gunggi = visitor[['경기도']]
visitor_gyungnam = visitor[['경상남도']]
visitor_gyungbuk = visitor[['경상북도']]
visitor_gangju = visitor[['광주광역시']]
visitor_daegu = visitor[['대구광역시']]
visitor_daejeon = visitor[['대전광역시']]
visitor_busan = visitor[['부산광역시']]
visitor_seoul = visitor[['서울특별시']]
visitor_ulsan = visitor[['울산광역시']]
visitor_incheon = visitor[['인천광역시']]
visitor_jeonnam = visitor[['전라남도']]
visitor_jeonbuk = visitor[['전라북도']]
visitor_jej = visitor[['제주특별자치도']]
visitor_chungnam = visitor[['충청남도']]
visitor_chungbuk = visitor[['충청북도']]

In [6]:
#충청북도 데이터 통합
total_chungbuk=pd.concat([chungbuk_df, gdp, cpi,expect_inflaction,  visitor_chungbuk], axis=1)
new_columns = ['호텔', '콘도', '캠핑장/펜션', '기타숙박', '면세점', '관광기념품', '레저용품쇼핑', '대형쇼핑몰', '관광유원시설', '골프장','스키장', '기타레저', '문화서비스', '식음료', 'gdp','CPI','기대 인플레이션율',  '충청북도 관광객']
total_chungbuk.columns = new_columns

In [7]:
#충청북도 통합 데이터에서 호텔 및 예측에 필요한 데이터만 추출
chungbuk_hotel=total_chungbuk[['호텔', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 콘도 및 예측에 필요한 데이터만 추출
chungbuk_condo=total_chungbuk[['콘도', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 캠핑장/펜션 및 예측에 필요한 데이터만 추출
chungbuk_camping=total_chungbuk[['캠핑장/펜션', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 기타숙박 및 예측에 필요한 데이터만 추출
chungbuk_elsesleep=total_chungbuk[['기타숙박', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 면세점 및 예측에 필요한 데이터만 추출
chungbuk_dutyfree=total_chungbuk[['면세점', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 관광기념품 및 예측에 필요한 데이터만 추출
chungbuk_souvenir=total_chungbuk[['관광기념품', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 레저용품쇼핑 및 예측에 필요한 데이터만 추출
chungbuk_leisureshop=total_chungbuk[['레저용품쇼핑', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 대형쇼핑몰 및 예측에 필요한 데이터만 추출
chungbuk_hugeshop=total_chungbuk[['대형쇼핑몰', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 관광유원시설 및 예측에 필요한 데이터만 추출
chungbuk_amuse=total_chungbuk[['관광유원시설', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 골프장 및 예측에 필요한 데이터만 추출
chungbuk_golf=total_chungbuk[['골프장', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 스키장 및 예측에 필요한 데이터만 추출
chungbuk_ski=total_chungbuk[['스키장', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 기타레저 및 예측에 필요한 데이터만 추출
chungbuk_leisure=total_chungbuk[['기타레저', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 문화서비스 및 예측에 필요한 데이터만 추출
chungbuk_culture=total_chungbuk[['문화서비스', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

#충청북도 통합 데이터에서 식음료 및 예측에 필요한 데이터만 추출
chungbuk_food=total_chungbuk[['식음료', 'gdp','CPI','기대 인플레이션율','충청북도 관광객']]

In [8]:
#충청북도 예측
# 호텔 예측 학습한 데이터를 다른 데이터로 평가하는 것예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_hotel = chungbuk_hotel[chungbuk_hotel.columns.difference(['호텔'])]
y_chungbuk_hotel = chungbuk_hotel['호텔']
X_chungbuk_hotel_train , X_chungbuk_hotel_test , y_chungbuk_hotel_train , y_chungbuk_hotel_test = train_test_split(X_chungbuk_hotel , y_chungbuk_hotel , test_size = 0.2 , random_state = 20)
lr1 = linear_model.LinearRegression()
model1 = lr1.fit(X_chungbuk_hotel_train , y_chungbuk_hotel_train)

#충청북도 예측
# condo 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_condo = chungbuk_condo[chungbuk_condo.columns.difference(['콘도'])]
y_chungbuk_condo = chungbuk_condo['콘도']
X_chungbuk_condo_train , X_chungbuk_condo_test , y_chungbuk_condo_train , y_chungbuk_condo_test = train_test_split(X_chungbuk_condo , y_chungbuk_condo , test_size = 0.2 , random_state = 20)
lr２ = linear_model.LinearRegression()
model２ = lr２.fit(X_chungbuk_condo_train , y_chungbuk_condo_train)

#충청북도 예측
# camping 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_camping = chungbuk_camping[chungbuk_camping.columns.difference(['캠핑장/펜션'])]
y_chungbuk_camping = chungbuk_camping['캠핑장/펜션']
X_chungbuk_camping_train , X_chungbuk_camping_test , y_chungbuk_camping_train , y_chungbuk_camping_test = train_test_split(X_chungbuk_camping , y_chungbuk_camping , test_size = 0.2 , random_state = 20)
lr3 = linear_model.LinearRegression()
model3 = lr3.fit(X_chungbuk_camping_train , y_chungbuk_camping_train)

#충청북도 예측
# elsesleep 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_elsesleep = chungbuk_elsesleep[chungbuk_elsesleep.columns.difference(['기타숙박'])]
y_chungbuk_elsesleep = chungbuk_elsesleep['기타숙박']
X_chungbuk_elsesleep_train , X_chungbuk_elsesleep_test , y_chungbuk_elsesleep_train , y_chungbuk_elsesleep_test = train_test_split(X_chungbuk_elsesleep , y_chungbuk_elsesleep , test_size = 0.2 , random_state = 20)
lr4 = linear_model.LinearRegression()
model4 = lr4.fit(X_chungbuk_elsesleep_train , y_chungbuk_elsesleep_train)

#충청북도 예측
# dutyfree 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_dutyfree = chungbuk_dutyfree[chungbuk_dutyfree.columns.difference(['면세점'])]
y_chungbuk_dutyfree = chungbuk_dutyfree['면세점']
X_chungbuk_dutyfree_train , X_chungbuk_dutyfree_test , y_chungbuk_dutyfree_train , y_chungbuk_dutyfree_test = train_test_split(X_chungbuk_dutyfree , y_chungbuk_dutyfree , test_size = 0.2 , random_state = 20)
lr5 = linear_model.LinearRegression()
model5 = lr5.fit(X_chungbuk_dutyfree_train , y_chungbuk_dutyfree_train)

#충청북도 예측
# 레저용품쇼핑 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_leisureshop = chungbuk_leisureshop[chungbuk_leisureshop.columns.difference(['레저용품쇼핑'])]
y_chungbuk_leisureshop = chungbuk_leisureshop['레저용품쇼핑']
X_chungbuk_leisureshop_train , X_chungbuk_leisureshop_test , y_chungbuk_leisureshop_train , y_chungbuk_leisureshop_test = train_test_split(X_chungbuk_leisureshop , y_chungbuk_leisureshop , test_size = 0.2 , random_state = 20)
lr6 = linear_model.LinearRegression()
model6 = lr6.fit(X_chungbuk_leisureshop_train , y_chungbuk_leisureshop_train)

#충청북도 예측
# hugeshop 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_hugeshop = chungbuk_hugeshop[chungbuk_hugeshop.columns.difference(['대형쇼핑몰'])]
y_chungbuk_hugeshop = chungbuk_hugeshop['대형쇼핑몰']
X_chungbuk_hugeshop_train , X_chungbuk_hugeshop_test , y_chungbuk_hugeshop_train , y_chungbuk_hugeshop_test = train_test_split(X_chungbuk_hugeshop , y_chungbuk_hugeshop , test_size = 0.2 , random_state = 20)
lr7 = linear_model.LinearRegression()
model7 = lr7.fit(X_chungbuk_hugeshop_train , y_chungbuk_hugeshop_train)

#충청북도 예측
# amuse 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_amuse = chungbuk_amuse[chungbuk_amuse.columns.difference(['관광유원시설'])]
y_chungbuk_amuse = chungbuk_amuse['관광유원시설']
X_chungbuk_amuse_train , X_chungbuk_amuse_test , y_chungbuk_amuse_train , y_chungbuk_amuse_test = train_test_split(X_chungbuk_amuse , y_chungbuk_amuse , test_size = 0.2 , random_state = 20)
lr8 = linear_model.LinearRegression()
model8 = lr8.fit(X_chungbuk_amuse_train , y_chungbuk_amuse_train)

#충청북도 예측
# golf 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_golf = chungbuk_golf[chungbuk_golf.columns.difference(['골프장'])]
y_chungbuk_golf = chungbuk_golf['골프장']
X_chungbuk_golf_train , X_chungbuk_golf_test , y_chungbuk_golf_train , y_chungbuk_golf_test = train_test_split(X_chungbuk_golf , y_chungbuk_golf , test_size = 0.2 , random_state = 20)
lr9 = linear_model.LinearRegression()
model9 = lr9.fit(X_chungbuk_golf_train , y_chungbuk_golf_train)

#충청북도 예측
# ski 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_ski = chungbuk_ski[chungbuk_ski.columns.difference(['스키장'])]
y_chungbuk_ski = chungbuk_ski['스키장']
X_chungbuk_ski_train , X_chungbuk_ski_test , y_chungbuk_ski_train , y_chungbuk_ski_test = train_test_split(X_chungbuk_ski , y_chungbuk_ski , test_size = 0.2 , random_state = 20)
lr10 = linear_model.LinearRegression()
model10 = lr10.fit(X_chungbuk_ski_train , y_chungbuk_ski_train)

#충청북도 예측
# culture 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_culture = chungbuk_culture[chungbuk_culture.columns.difference(['문화서비스'])]
y_chungbuk_culture = chungbuk_culture['문화서비스']
X_chungbuk_culture_train , X_chungbuk_culture_test , y_chungbuk_culture_train , y_chungbuk_culture_test = train_test_split(X_chungbuk_culture , y_chungbuk_culture , test_size = 0.2 , random_state = 20)
lr12 = linear_model.LinearRegression()
model12 = lr12.fit(X_chungbuk_culture_train , y_chungbuk_culture_train)

#충청북도 예측
# food 예측 학습한 데이터를 다른 데이터로 평가하는 것 예측 테스트(X축에는 예측할 데이터, Y축에는 예측에 고려할 데이터)
X_chungbuk_food = chungbuk_food[chungbuk_food.columns.difference(['식음료'])]
y_chungbuk_food = chungbuk_food['식음료']
X_chungbuk_food_train , X_chungbuk_food_test , y_chungbuk_food_train , y_chungbuk_food_test = train_test_split(X_chungbuk_food , y_chungbuk_food , test_size = 0.2 , random_state = 20)
lr13 = linear_model.LinearRegression()
model13 = lr13.fit(X_chungbuk_food_train , y_chungbuk_food_train)

In [9]:
# 충청북도 예측에 고려할 공통 데이터
#GDP 상승률 1.4% 가정
gdp_2024=gdp.iloc[-12:]
gdp_2024=gdp_2024*1.014
new_columns = ['2024년 01월', '2024년 02월', '2024년 03월' , '2024년 04월', '2024년 05월', '2024년 06월', '2024년 07월', '2024년 08월', '2024년 09월', '2024년 10월', '2024년 11월', '2024년 12월']
gdp_2024.index = new_columns

#CPI 상승률 3.1 가정
cpi_2024=cpi.iloc[-12:]
cpi_2024=cpi_2024*1.031
new_columns = ['2024년 01월', '2024년 02월', '2024년 03월' , '2024년 04월', '2024년 05월', '2024년 06월', '2024년 07월', '2024년 08월', '2024년 09월', '2024년 10월', '2024년 11월', '2024년 12월']
cpi_2024.index = new_columns

#기대인플레이션율 상승률 2% 가정
expect_inflaction_2024=expect_inflaction.iloc[-12:]
expect_inflaction_2024=expect_inflaction_2024*1.031
new_columns = ['2024년 01월', '2024년 02월', '2024년 03월' , '2024년 04월', '2024년 05월', '2024년 06월', '2024년 07월', '2024년 08월', '2024년 09월', '2024년 10월', '2024년 11월', '2024년 12월']
expect_inflaction_2024.index = new_columns

#관광객 상승률 8% 가정
visitor_chungbuk_2024=visitor_chungbuk.iloc[-12:]
visitor_chungbuk_2024=visitor_chungbuk_2024*1.08
new_columns = ['2024년 01월', '2024년 02월', '2024년 03월' , '2024년 04월', '2024년 05월', '2024년 06월', '2024년 07월', '2024년 08월', '2024년 09월', '2024년 10월', '2024년 11월', '2024년 12월']
visitor_chungbuk_2024.index = new_columns

# Y축 데이터 통합(충청북도_호텔)
chungbuk_practice = pd.concat([gdp_2024, cpi_2024,expect_inflaction_2024, visitor_chungbuk_2024], axis=1)
new_columns = ['CPI', 'gdp', '기대 인플레이션율', '충청북도 관광객']
chungbuk_practice.columns = new_columns

In [10]:
# 스케일링할 데이터 선택
X_practice = chungbuk_practice.values

# 스케일러 객체 생성
scaler = StandardScaler()

# 스케일링 적용
X_practice_scaled = scaler.fit_transform(X_practice)

# 스케일링된 데이터를 DataFrame으로 변환
chungbuk_practice_scaled = pd.DataFrame(X_practice_scaled, columns=chungbuk_practice.columns, index=chungbuk_practice.index)

In [11]:
# 호텔 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_hotel.columns.difference(['호텔'])]

# 호텔 소비 예측
predict_hotel = lr1.predict(X_practice)
predict_hotel = np.round(predict_hotel, decimals=1)
predict_hotel[predict_hotel < 0] = -predict_hotel[predict_hotel < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_hotel, columns=['2024년 충청북도 호텔 소비 예측치'], index=chungbuk_practice.index)

# 결과 출력
predict



#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_hotel_24_jan=predict_hotel[0]
chungbuk_hotel_24_feb=predict_hotel[1]
chungbuk_hotel_24_mar=predict_hotel[2]
chungbuk_hotel_24_apr=predict_hotel[3]
chungbuk_hotel_24_may=predict_hotel[4]
chungbuk_hotel_24_jun=predict_hotel[5]
chungbuk_hotel_24_jul=predict_hotel[6]
chungbuk_hotel_24_aug=predict_hotel[7]
chungbuk_hotel_24_sep=predict_hotel[8]
chungbuk_hotel_24_oct=predict_hotel[9]
chungbuk_hotel_24_nov=predict_hotel[10]
chungbuk_hotel_24_dec=predict_hotel[11]

chungbuk_hotel_24 = {
    "2024년 01월 충청북도 호텔": chungbuk_hotel_24_jan,
    "2024년 02월 충청북도 호텔": chungbuk_hotel_24_feb,
    "2024년 03월 충청북도 호텔": chungbuk_hotel_24_mar,
    "2024년 04월 충청북도 호텔": chungbuk_hotel_24_apr,
    "2024년 05월 충청북도 호텔": chungbuk_hotel_24_may,
    "2024년 06월 충청북도 호텔": chungbuk_hotel_24_jun,
    "2024년 07월 충청북도 호텔": chungbuk_hotel_24_jul,
    "2024년 08월 충청북도 호텔": chungbuk_hotel_24_aug,
    "2024년 09월 충청북도 호텔": chungbuk_hotel_24_sep,
    "2024년 10월 충청북도 호텔": chungbuk_hotel_24_oct,
    "2024년 11월 충청북도 호텔": chungbuk_hotel_24_nov,
    "2024년 12월 충청북도 호텔": chungbuk_hotel_24_dec
}

# 콘도 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_condo.columns.difference(['콘도'])]

# 콘도 소비 예측
predict_condo = lr2.predict(X_practice)
predict_condo = np.round(predict_condo, decimals=1)
predict_condo[predict_condo < 0] = -predict_condo[predict_condo < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_condo, columns=['2024년 충청북도 콘도 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_condo_24_jan=predict_condo[0]
chungbuk_condo_24_feb=predict_condo[1]
chungbuk_condo_24_mar=predict_condo[2]
chungbuk_condo_24_apr=predict_condo[3]
chungbuk_condo_24_may=predict_condo[4]
chungbuk_condo_24_jun=predict_condo[5]
chungbuk_condo_24_jul=predict_condo[6]
chungbuk_condo_24_aug=predict_condo[7]
chungbuk_condo_24_sep=predict_condo[8]
chungbuk_condo_24_oct=predict_condo[9]
chungbuk_condo_24_nov=predict_condo[10]
chungbuk_condo_24_dec=predict_condo[11]

chungbuk_condo_24 = {
    "2024년 01월 충청북도 콘도": chungbuk_condo_24_jan,
    "2024년 02월 충청북도 콘도": chungbuk_condo_24_feb,
    "2024년 03월 충청북도 콘도": chungbuk_condo_24_mar,
    "2024년 04월 충청북도 콘도": chungbuk_condo_24_apr,
    "2024년 05월 충청북도 콘도": chungbuk_condo_24_may,
    "2024년 06월 충청북도 콘도": chungbuk_condo_24_jun,
    "2024년 07월 충청북도 콘도": chungbuk_condo_24_jul,
    "2024년 08월 충청북도 콘도": chungbuk_condo_24_aug,
    "2024년 09월 충청북도 콘도": chungbuk_condo_24_sep,
    "2024년 10월 충청북도 콘도": chungbuk_condo_24_oct,
    "2024년 11월 충청북도 콘도": chungbuk_condo_24_nov,
    "2024년 12월 충청북도 콘도": chungbuk_condo_24_dec
}

# 캠핑장/펜션 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_camping.columns.difference(['캠핑장/펜션'])]

# 캠핑장/펜션 소비 예측
predict_camping = lr3.predict(X_practice)
predict_camping = np.round(predict_camping, decimals=1)
predict_camping[predict_camping < 0] = -predict_camping[predict_camping < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_camping, columns=['2024년 충청북도 캠핑장/펜션 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_camping_24_jan=predict_camping[0]
chungbuk_camping_24_feb=predict_camping[1]
chungbuk_camping_24_mar=predict_camping[2]
chungbuk_camping_24_apr=predict_camping[3]
chungbuk_camping_24_may=predict_camping[4]
chungbuk_camping_24_jun=predict_camping[5]
chungbuk_camping_24_jul=predict_camping[6]
chungbuk_camping_24_aug=predict_camping[7]
chungbuk_camping_24_sep=predict_camping[8]
chungbuk_camping_24_oct=predict_camping[9]
chungbuk_camping_24_nov=predict_camping[10]
chungbuk_camping_24_dec=predict_camping[11]

chungbuk_camping_24 = {
    "2024년 01월 충청북도 캠핑장/펜션": chungbuk_camping_24_jan,
    "2024년 02월 충청북도 캠핑장/펜션": chungbuk_camping_24_feb,
    "2024년 03월 충청북도 캠핑장/펜션": chungbuk_camping_24_mar,
    "2024년 04월 충청북도 캠핑장/펜션": chungbuk_camping_24_apr,
    "2024년 05월 충청북도 캠핑장/펜션": chungbuk_camping_24_may,
    "2024년 06월 충청북도 캠핑장/펜션": chungbuk_camping_24_jun,
    "2024년 07월 충청북도 캠핑장/펜션": chungbuk_camping_24_jul,
    "2024년 08월 충청북도 캠핑장/펜션": chungbuk_camping_24_aug,
    "2024년 09월 충청북도 캠핑장/펜션": chungbuk_camping_24_sep,
    "2024년 10월 충청북도 캠핑장/펜션": chungbuk_camping_24_oct,
    "2024년 11월 충청북도 캠핑장/펜션": chungbuk_camping_24_nov,
    "2024년 12월 충청북도 캠핑장/펜션": chungbuk_camping_24_dec
}

# 기타숙박 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_elsesleep.columns.difference(['기타숙박'])]

# 기타숙박 소비 예측
predict_elsesleep = lr4.predict(X_practice)
predict_elsesleep = np.round(predict_elsesleep, decimals=1)
predict_elsesleep[predict_elsesleep < 0] = -predict_elsesleep[predict_elsesleep < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_elsesleep, columns=['2024년 충청북도 기타숙박 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_elsesleep_24_jan=predict_elsesleep[0]
chungbuk_elsesleep_24_feb=predict_elsesleep[1]
chungbuk_elsesleep_24_mar=predict_elsesleep[2]
chungbuk_elsesleep_24_apr=predict_elsesleep[3]
chungbuk_elsesleep_24_may=predict_elsesleep[4]
chungbuk_elsesleep_24_jun=predict_elsesleep[5]
chungbuk_elsesleep_24_jul=predict_elsesleep[6]
chungbuk_elsesleep_24_aug=predict_elsesleep[7]
chungbuk_elsesleep_24_sep=predict_elsesleep[8]
chungbuk_elsesleep_24_oct=predict_elsesleep[9]
chungbuk_elsesleep_24_nov=predict_elsesleep[10]
chungbuk_elsesleep_24_dec=predict_elsesleep[11]

chungbuk_elsesleep_24 = {
    "2024년 01월 충청북도 기타숙박": chungbuk_elsesleep_24_jan,
    "2024년 02월 충청북도 기타숙박": chungbuk_elsesleep_24_feb,
    "2024년 03월 충청북도 기타숙박": chungbuk_elsesleep_24_mar,
    "2024년 04월 충청북도 기타숙박": chungbuk_elsesleep_24_apr,
    "2024년 05월 충청북도 기타숙박": chungbuk_elsesleep_24_may,
    "2024년 06월 충청북도 기타숙박": chungbuk_elsesleep_24_jun,
    "2024년 07월 충청북도 기타숙박": chungbuk_elsesleep_24_jul,
    "2024년 08월 충청북도 기타숙박": chungbuk_elsesleep_24_aug,
    "2024년 09월 충청북도 기타숙박": chungbuk_elsesleep_24_sep,
    "2024년 10월 충청북도 기타숙박": chungbuk_elsesleep_24_oct,
    "2024년 11월 충청북도 기타숙박": chungbuk_elsesleep_24_nov,
    "2024년 12월 충청북도 기타숙박": chungbuk_elsesleep_24_dec
}

# 면세점 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_dutyfree.columns.difference(['면세점'])]

# 면세점 소비 예측
predict_dutyfree = lr5.predict(X_practice)
predict_dutyfree = np.round(predict_dutyfree, decimals=1)
predict_dutyfree[predict_dutyfree < 0] = -predict_dutyfree[predict_dutyfree < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_dutyfree, columns=['2024년 충청북도 면세점 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_dutyfree_24_jan=predict_dutyfree[0]
chungbuk_dutyfree_24_feb=predict_dutyfree[1]
chungbuk_dutyfree_24_mar=predict_dutyfree[2]
chungbuk_dutyfree_24_apr=predict_dutyfree[3]
chungbuk_dutyfree_24_may=predict_dutyfree[4]
chungbuk_dutyfree_24_jun=predict_dutyfree[5]
chungbuk_dutyfree_24_jul=predict_dutyfree[6]
chungbuk_dutyfree_24_aug=predict_dutyfree[7]
chungbuk_dutyfree_24_sep=predict_dutyfree[8]
chungbuk_dutyfree_24_oct=predict_dutyfree[9]
chungbuk_dutyfree_24_nov=predict_dutyfree[10]
chungbuk_dutyfree_24_dec=predict_dutyfree[11]

chungbuk_dutyfree_24 = {
    "2024년 01월 충청북도 면세점": chungbuk_dutyfree_24_jan,
    "2024년 02월 충청북도 면세점": chungbuk_dutyfree_24_feb,
    "2024년 03월 충청북도 면세점": chungbuk_dutyfree_24_mar,
    "2024년 04월 충청북도 면세점": chungbuk_dutyfree_24_apr,
    "2024년 05월 충청북도 면세점": chungbuk_dutyfree_24_may,
    "2024년 06월 충청북도 면세점": chungbuk_dutyfree_24_jun,
    "2024년 07월 충청북도 면세점": chungbuk_dutyfree_24_jul,
    "2024년 08월 충청북도 면세점": chungbuk_dutyfree_24_aug,
    "2024년 09월 충청북도 면세점": chungbuk_dutyfree_24_sep,
    "2024년 10월 충청북도 면세점": chungbuk_dutyfree_24_oct,
    "2024년 11월 충청북도 면세점": chungbuk_dutyfree_24_nov,
    "2024년 12월 충청북도 면세점": chungbuk_dutyfree_24_dec
}

# 레저용품쇼핑 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_leisureshop.columns.difference(['레저용품쇼핑'])]

# 레저용품쇼핑 소비 예측
predict_leisureshop = lr6.predict(X_practice)
predict_leisureshop = np.round(predict_leisureshop, decimals=1)
predict_leisureshop[predict_leisureshop < 0] = -predict_leisureshop[predict_leisureshop < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_leisureshop, columns=['2024년 충청북도 레저용품쇼핑 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_leisureshop_24_jan=predict_leisureshop[0]
chungbuk_leisureshop_24_feb=predict_leisureshop[1]
chungbuk_leisureshop_24_mar=predict_leisureshop[2]
chungbuk_leisureshop_24_apr=predict_leisureshop[3]
chungbuk_leisureshop_24_may=predict_leisureshop[4]
chungbuk_leisureshop_24_jun=predict_leisureshop[5]
chungbuk_leisureshop_24_jul=predict_leisureshop[6]
chungbuk_leisureshop_24_aug=predict_leisureshop[7]
chungbuk_leisureshop_24_sep=predict_leisureshop[8]
chungbuk_leisureshop_24_oct=predict_leisureshop[9]
chungbuk_leisureshop_24_nov=predict_leisureshop[10]
chungbuk_leisureshop_24_dec=predict_leisureshop[11]

chungbuk_leisureshop_24 = {
    "2024년 01월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_jan,
    "2024년 02월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_feb,
    "2024년 03월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_mar,
    "2024년 04월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_apr,
    "2024년 05월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_may,
    "2024년 06월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_jun,
    "2024년 07월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_jul,
    "2024년 08월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_aug,
    "2024년 09월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_sep,
    "2024년 10월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_oct,
    "2024년 11월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_nov,
    "2024년 12월 충청북도 레저용품쇼핑": chungbuk_leisureshop_24_dec
}

# 대형쇼핑몰 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_hugeshop.columns.difference(['대형쇼핑몰'])]

# 대형쇼핑몰 소비 예측
predict_hugeshop = lr7.predict(X_practice)
predict_hugeshop = np.round(predict_hugeshop, decimals=1)
predict_hugeshop[predict_hugeshop < 0] = -predict_hugeshop[predict_hugeshop < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_hugeshop, columns=['2024년 충청북도 대형쇼핑몰 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_hugeshop_24_jan=predict_hugeshop[0]
chungbuk_hugeshop_24_feb=predict_hugeshop[1]
chungbuk_hugeshop_24_mar=predict_hugeshop[2]
chungbuk_hugeshop_24_apr=predict_hugeshop[3]
chungbuk_hugeshop_24_may=predict_hugeshop[4]
chungbuk_hugeshop_24_jun=predict_hugeshop[5]
chungbuk_hugeshop_24_jul=predict_hugeshop[6]
chungbuk_hugeshop_24_aug=predict_hugeshop[7]
chungbuk_hugeshop_24_sep=predict_hugeshop[8]
chungbuk_hugeshop_24_oct=predict_hugeshop[9]
chungbuk_hugeshop_24_nov=predict_hugeshop[10]
chungbuk_hugeshop_24_dec=predict_hugeshop[11]

chungbuk_hugeshop_24 = {
    "2024년 01월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_jan,
    "2024년 02월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_feb,
    "2024년 03월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_mar,
    "2024년 04월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_apr,
    "2024년 05월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_may,
    "2024년 06월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_jun,
    "2024년 07월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_jul,
    "2024년 08월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_aug,
    "2024년 09월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_sep,
    "2024년 10월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_oct,
    "2024년 11월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_nov,
    "2024년 12월 충청북도 대형쇼핑몰": chungbuk_hugeshop_24_dec
}

# 관광유원시설 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_amuse.columns.difference(['관광유원시설'])]

# 관광유원시설 소비 예측
predict_amuse = lr8.predict(X_practice)
predict_amuse = np.round(predict_amuse, decimals=1)
predict_amuse[predict_amuse < 0] = -predict_amuse[predict_amuse < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_amuse, columns=['2024년 충청북도 관광유원시설 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_amuse_24_jan=predict_amuse[0]
chungbuk_amuse_24_feb=predict_amuse[1]
chungbuk_amuse_24_mar=predict_amuse[2]
chungbuk_amuse_24_apr=predict_amuse[3]
chungbuk_amuse_24_may=predict_amuse[4]
chungbuk_amuse_24_jun=predict_amuse[5]
chungbuk_amuse_24_jul=predict_amuse[6]
chungbuk_amuse_24_aug=predict_amuse[7]
chungbuk_amuse_24_sep=predict_amuse[8]
chungbuk_amuse_24_oct=predict_amuse[9]
chungbuk_amuse_24_nov=predict_amuse[10]
chungbuk_amuse_24_dec=predict_amuse[11]

chungbuk_amuse_24 = {
    "2024년 01월 충청북도 관광유원시설": chungbuk_amuse_24_jan,
    "2024년 02월 충청북도 관광유원시설": chungbuk_amuse_24_feb,
    "2024년 03월 충청북도 관광유원시설": chungbuk_amuse_24_mar,
    "2024년 04월 충청북도 관광유원시설": chungbuk_amuse_24_apr,
    "2024년 05월 충청북도 관광유원시설": chungbuk_amuse_24_may,
    "2024년 06월 충청북도 관광유원시설": chungbuk_amuse_24_jun,
    "2024년 07월 충청북도 관광유원시설": chungbuk_amuse_24_jul,
    "2024년 08월 충청북도 관광유원시설": chungbuk_amuse_24_aug,
    "2024년 09월 충청북도 관광유원시설": chungbuk_amuse_24_sep,
    "2024년 10월 충청북도 관광유원시설": chungbuk_amuse_24_oct,
    "2024년 11월 충청북도 관광유원시설": chungbuk_amuse_24_nov,
    "2024년 12월 충청북도 관광유원시설": chungbuk_amuse_24_dec
}

# 골프장 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_golf.columns.difference(['골프장'])]

# 골프장 소비 예측
predict_golf = lr9.predict(X_practice)
predict_golf = np.round(predict_golf, decimals=1)
predict_golf[predict_golf < 0] = -predict_golf[predict_golf < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_golf, columns=['2024년 충청북도 골프장 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_golf_24_jan=predict_golf[0]
chungbuk_golf_24_feb=predict_golf[1]
chungbuk_golf_24_mar=predict_golf[2]
chungbuk_golf_24_apr=predict_golf[3]
chungbuk_golf_24_may=predict_golf[4]
chungbuk_golf_24_jun=predict_golf[5]
chungbuk_golf_24_jul=predict_golf[6]
chungbuk_golf_24_aug=predict_golf[7]
chungbuk_golf_24_sep=predict_golf[8]
chungbuk_golf_24_oct=predict_golf[9]
chungbuk_golf_24_nov=predict_golf[10]
chungbuk_golf_24_dec=predict_golf[11]

chungbuk_golf_24 = {
    "2024년 01월 충청북도 골프장": chungbuk_golf_24_jan,
    "2024년 02월 충청북도 골프장": chungbuk_golf_24_feb,
    "2024년 03월 충청북도 골프장": chungbuk_golf_24_mar,
    "2024년 04월 충청북도 골프장": chungbuk_golf_24_apr,
    "2024년 05월 충청북도 골프장": chungbuk_golf_24_may,
    "2024년 06월 충청북도 골프장": chungbuk_golf_24_jun,
    "2024년 07월 충청북도 골프장": chungbuk_golf_24_jul,
    "2024년 08월 충청북도 골프장": chungbuk_golf_24_aug,
    "2024년 09월 충청북도 골프장": chungbuk_golf_24_sep,
    "2024년 10월 충청북도 골프장": chungbuk_golf_24_oct,
    "2024년 11월 충청북도 골프장": chungbuk_golf_24_nov,
    "2024년 12월 충청북도 골프장": chungbuk_golf_24_dec
}

# 스키장 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_ski.columns.difference(['스키장'])]

# 스키장 소비 예측
predict_ski = lr10.predict(X_practice)
predict_ski = np.round(predict_ski, decimals=1)
predict_ski[predict_ski < 0] = -predict_ski[predict_ski < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_ski, columns=['2024년 충청북도 스키장 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_ski_24_jan=predict_ski[0]
chungbuk_ski_24_feb=predict_ski[1]
chungbuk_ski_24_mar=predict_ski[2]
chungbuk_ski_24_apr=predict_ski[3]
chungbuk_ski_24_may=predict_ski[4]
chungbuk_ski_24_jun=predict_ski[5]
chungbuk_ski_24_jul=predict_ski[6]
chungbuk_ski_24_aug=predict_ski[7]
chungbuk_ski_24_sep=predict_ski[8]
chungbuk_ski_24_oct=predict_ski[9]
chungbuk_ski_24_nov=predict_ski[10]
chungbuk_ski_24_dec=predict_ski[11]

chungbuk_ski_24 = {
    "2024년 01월 충청북도 스키장": chungbuk_ski_24_jan,
    "2024년 02월 충청북도 스키장": chungbuk_ski_24_feb,
    "2024년 03월 충청북도 스키장": chungbuk_ski_24_mar,
    "2024년 04월 충청북도 스키장": chungbuk_ski_24_apr,
    "2024년 05월 충청북도 스키장": chungbuk_ski_24_may,
    "2024년 06월 충청북도 스키장": chungbuk_ski_24_jun,
    "2024년 07월 충청북도 스키장": chungbuk_ski_24_jul,
    "2024년 08월 충청북도 스키장": chungbuk_ski_24_aug,
    "2024년 09월 충청북도 스키장": chungbuk_ski_24_sep,
    "2024년 10월 충청북도 스키장": chungbuk_ski_24_oct,
    "2024년 11월 충청북도 스키장": chungbuk_ski_24_nov,
    "2024년 12월 충청북도 스키장": chungbuk_ski_24_dec
}

# 문화서비스 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_culture.columns.difference(['문화서비스'])]

# 문화서비스 소비 예측
predict_culture = lr12.predict(X_practice)
predict_culture = np.round(predict_culture, decimals=1)
predict_culture[predict_culture < 0] = -predict_culture[predict_culture < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_culture, columns=['2024년 충청북도 문화서비스 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_culture_24_jan=predict_culture[0]
chungbuk_culture_24_feb=predict_culture[1]
chungbuk_culture_24_mar=predict_culture[2]
chungbuk_culture_24_apr=predict_culture[3]
chungbuk_culture_24_may=predict_culture[4]
chungbuk_culture_24_jun=predict_culture[5]
chungbuk_culture_24_jul=predict_culture[6]
chungbuk_culture_24_aug=predict_culture[7]
chungbuk_culture_24_sep=predict_culture[8]
chungbuk_culture_24_oct=predict_culture[9]
chungbuk_culture_24_nov=predict_culture[10]
chungbuk_culture_24_dec=predict_culture[11]

chungbuk_culture_24 = {
    "2024년 01월 충청북도 문화서비스": chungbuk_culture_24_jan,
    "2024년 02월 충청북도 문화서비스": chungbuk_culture_24_feb,
    "2024년 03월 충청북도 문화서비스": chungbuk_culture_24_mar,
    "2024년 04월 충청북도 문화서비스": chungbuk_culture_24_apr,
    "2024년 05월 충청북도 문화서비스": chungbuk_culture_24_may,
    "2024년 06월 충청북도 문화서비스": chungbuk_culture_24_jun,
    "2024년 07월 충청북도 문화서비스": chungbuk_culture_24_jul,
    "2024년 08월 충청북도 문화서비스": chungbuk_culture_24_aug,
    "2024년 09월 충청북도 문화서비스": chungbuk_culture_24_sep,
    "2024년 10월 충청북도 문화서비스": chungbuk_culture_24_oct,
    "2024년 11월 충청북도 문화서비스": chungbuk_culture_24_nov,
    "2024년 12월 충청북도 문화서비스": chungbuk_culture_24_dec
}

# 식음료 예측 결과치
X = chungbuk_practice_scaled

# 학습할 때 사용한 피처 순서와 동일하게 예측할 피처를 선택
X_practice = X[chungbuk_food.columns.difference(['식음료'])]

# 식음료 소비 예측
predict_food = lr13.predict(X_practice)
predict_food = np.round(predict_food, decimals=1)
predict_food[predict_food < 0] = -predict_food[predict_food < 0]

# 예측 결과를 DataFrame으로 변환
predict = pd.DataFrame(predict_food, columns=['2024년 충청북도 식음료 소비 예측치'], index=chungbuk_practice.index)


#1월 ~ 12월 예측 데이터 각 변수에 저장
chungbuk_food_24_jan=predict_food[0]
chungbuk_food_24_feb=predict_food[1]
chungbuk_food_24_mar=predict_food[2]
chungbuk_food_24_apr=predict_food[3]
chungbuk_food_24_may=predict_food[4]
chungbuk_food_24_jun=predict_food[5]
chungbuk_food_24_jul=predict_food[6]
chungbuk_food_24_aug=predict_food[7]
chungbuk_food_24_sep=predict_food[8]
chungbuk_food_24_oct=predict_food[9]
chungbuk_food_24_nov=predict_food[10]
chungbuk_food_24_dec=predict_food[11]

chungbuk_food_24 = {
    "2024년 01월 충청북도 식음료": chungbuk_food_24_jan,
    "2024년 02월 충청북도 식음료": chungbuk_food_24_feb,
    "2024년 03월 충청북도 식음료": chungbuk_food_24_mar,
    "2024년 04월 충청북도 식음료": chungbuk_food_24_apr,
    "2024년 05월 충청북도 식음료": chungbuk_food_24_may,
    "2024년 06월 충청북도 식음료": chungbuk_food_24_jun,
    "2024년 07월 충청북도 식음료": chungbuk_food_24_jul,
    "2024년 08월 충청북도 식음료": chungbuk_food_24_aug,
    "2024년 09월 충청북도 식음료": chungbuk_food_24_sep,
    "2024년 10월 충청북도 식음료": chungbuk_food_24_oct,
    "2024년 11월 충청북도 식음료": chungbuk_food_24_nov,
    "2024년 12월 충청북도 식음료": chungbuk_food_24_dec
}

In [12]:
chungbuk_hotel_24 = [{
    "predict_month": "2024년01월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_jan
}, {
    "predict_month": "2024년02월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_feb
},
 {
    "predict_month": "2024년03월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_mar
},
              
   {
    "predict_month": "2024년04월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_apr
},
{
    "predict_month": "2024년05월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_may
},
 {
    "predict_month": "2024년06월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_jun
},
 {
    "predict_month": "2024년07월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_jul
},
{
    "predict_month": "2024년08월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_aug
},
 {
    "predict_month": "2024년09월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_sep
},

              {
    "predict_month": "2024년10월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_oct
},
{
    "predict_month": "2024년11월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_nov
},
 {
    "predict_month": "2024년12월_hotel_chungbuk",
    "predict_value": chungbuk_hotel_24_dec
},
        
]

chungbuk_condo_24 = [{
    "predict_month": "2024년01월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_jan
}, {
    "predict_month": "2024년02월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_feb
},
 {
    "predict_month": "2024년03월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_mar
},
              
   {
    "predict_month": "2024년04월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_apr
},
{
    "predict_month": "2024년05월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_may
},
 {
    "predict_month": "2024년06월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_jun
},
 {
    "predict_month": "2024년07월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_jul
},
{
    "predict_month": "2024년08월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_aug
},
 {
    "predict_month": "2024년09월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_sep
},

              {
    "predict_month": "2024년10월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_oct
},
{
    "predict_month": "2024년11월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_nov
},
 {
    "predict_month": "2024년12월_condo_chungbuk",
    "predict_value": chungbuk_condo_24_dec
},
        
]

chungbuk_camping_24 = [{
    "predict_month": "2024년01월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_jan
}, {
    "predict_month": "2024년02월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_feb
},
 {
    "predict_month": "2024년03월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_mar
},
              
   {
    "predict_month": "2024년04월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_apr
},
{
    "predict_month": "2024년05월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_may
},
 {
    "predict_month": "2024년06월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_jun
},
 {
    "predict_month": "2024년07월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_jul
},
{
    "predict_month": "2024년08월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_aug
},
 {
    "predict_month": "2024년09월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_sep
},

              {
    "predict_month": "2024년10월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_oct
},
{
    "predict_month": "2024년11월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_nov
},
 {
    "predict_month": "2024년12월_camping_chungbuk",
    "predict_value": chungbuk_camping_24_dec
},
        
]

chungbuk_elsesleep_24 = [{
    "predict_month": "2024년01월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_jan
}, {
    "predict_month": "2024년02월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_feb
},
 {
    "predict_month": "2024년03월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_mar
},
              
   {
    "predict_month": "2024년04월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_apr
},
{
    "predict_month": "2024년05월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_may
},
 {
    "predict_month": "2024년06월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_jun
},
 {
    "predict_month": "2024년07월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_jul
},
{
    "predict_month": "2024년08월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_aug
},
 {
    "predict_month": "2024년09월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_sep
},

              {
    "predict_month": "2024년10월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_oct
},
{
    "predict_month": "2024년11월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_nov
},
 {
    "predict_month": "2024년12월_elsesleep_chungbuk",
    "predict_value": chungbuk_elsesleep_24_dec
},
        
]

chungbuk_dutyfree_24 = [{
    "predict_month": "2024년01월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_jan
}, {
    "predict_month": "2024년02월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_feb
},
 {
    "predict_month": "2024년03월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_mar
},
              
   {
    "predict_month": "2024년04월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_apr
},
{
    "predict_month": "2024년05월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_may
},
 {
    "predict_month": "2024년06월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_jun
},
 {
    "predict_month": "2024년07월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_jul
},
{
    "predict_month": "2024년08월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_aug
},
 {
    "predict_month": "2024년09월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_sep
},

              {
    "predict_month": "2024년10월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_oct
},
{
    "predict_month": "2024년11월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_nov
},
 {
    "predict_month": "2024년12월_dutyfree_chungbuk",
    "predict_value": chungbuk_dutyfree_24_dec
},
        
]

chungbuk_leisureshop_24 = [{
    "predict_month": "2024년01월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_jan
}, {
    "predict_month": "2024년02월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_feb
},
 {
    "predict_month": "2024년03월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_mar
},
              
   {
    "predict_month": "2024년04월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_apr
},
{
    "predict_month": "2024년05월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_may
},
 {
    "predict_month": "2024년06월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_jun
},
 {
    "predict_month": "2024년07월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_jul
},
{
    "predict_month": "2024년08월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_aug
},
 {
    "predict_month": "2024년09월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_sep
},

              {
    "predict_month": "2024년10월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_oct
},
{
    "predict_month": "2024년11월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_nov
},
 {
    "predict_month": "2024년12월_leisureshop_chungbuk",
    "predict_value": chungbuk_leisureshop_24_dec
},
        
]

chungbuk_hugeshop_24 = [{
    "predict_month": "2024년01월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_jan
}, {
    "predict_month": "2024년02월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_feb
},
 {
    "predict_month": "2024년03월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_mar
},
              
   {
    "predict_month": "2024년04월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_apr
},
{
    "predict_month": "2024년05월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_may
},
 {
    "predict_month": "2024년06월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_jun
},
 {
    "predict_month": "2024년07월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_jul
},
{
    "predict_month": "2024년08월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_aug
},
 {
    "predict_month": "2024년09월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_sep
},

              {
    "predict_month": "2024년10월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_oct
},
{
    "predict_month": "2024년11월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_nov
},
 {
    "predict_month": "2024년12월_hugeshop_chungbuk",
    "predict_value": chungbuk_hugeshop_24_dec
},
        
]

chungbuk_amuse_24 = [{
    "predict_month": "2024년01월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_jan
}, {
    "predict_month": "2024년02월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_feb
},
 {
    "predict_month": "2024년03월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_mar
},
              
   {
    "predict_month": "2024년04월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_apr
},
{
    "predict_month": "2024년05월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_may
},
 {
    "predict_month": "2024년06월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_jun
},
 {
    "predict_month": "2024년07월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_jul
},
{
    "predict_month": "2024년08월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_aug
},
 {
    "predict_month": "2024년09월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_sep
},

              {
    "predict_month": "2024년10월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_oct
},
{
    "predict_month": "2024년11월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_nov
},
 {
    "predict_month": "2024년12월_amuse_chungbuk",
    "predict_value": chungbuk_amuse_24_dec
},
        
]

chungbuk_golf_24 = [{
    "predict_month": "2024년01월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_jan
}, {
    "predict_month": "2024년02월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_feb
},
 {
    "predict_month": "2024년03월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_mar
},
              
   {
    "predict_month": "2024년04월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_apr
},
{
    "predict_month": "2024년05월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_may
},
 {
    "predict_month": "2024년06월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_jun
},
 {
    "predict_month": "2024년07월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_jul
},
{
    "predict_month": "2024년08월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_aug
},
 {
    "predict_month": "2024년09월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_sep
},

         {
    "predict_month": "2024년10월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_oct
},
{
    "predict_month": "2024년11월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_nov
},
 {
    "predict_month": "2024년12월_golf_chungbuk",
    "predict_value": chungbuk_golf_24_dec
},
        
]
chungbuk_ski_24 = [{
    "predict_month": "2024년01월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_jan
}, {
    "predict_month": "2024년02월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_feb
},
 {
    "predict_month": "2024년03월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_mar
},
              
   {
    "predict_month": "2024년04월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_apr
},
{
    "predict_month": "2024년05월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_may
},
 {
    "predict_month": "2024년06월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_jun
},
 {
    "predict_month": "2024년07월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_jul
},
{
    "predict_month": "2024년08월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_aug
},
 {
    "predict_month": "2024년09월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_sep
},

         {
    "predict_month": "2024년10월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_oct
},
{
    "predict_month": "2024년11월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_nov
},
 {
    "predict_month": "2024년12월_ski_chungbuk",
    "predict_value": chungbuk_ski_24_dec
},
        
]

chungbuk_culture_24 = [{
    "predict_month": "2024년01월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_jan
}, {
    "predict_month": "2024년02월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_feb
},
 {
    "predict_month": "2024년03월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_mar
},
              
   {
    "predict_month": "2024년04월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_apr
},
{
    "predict_month": "2024년05월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_may
},
 {
    "predict_month": "2024년06월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_jun
},
 {
    "predict_month": "2024년07월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_jul
},
{
    "predict_month": "2024년08월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_aug
},
 {
    "predict_month": "2024년09월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_sep
},

         {
    "predict_month": "2024년10월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_oct
},
{
    "predict_month": "2024년11월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_nov
},
 {
    "predict_month": "2024년12월_culture_chungbuk",
    "predict_value": chungbuk_culture_24_dec
},
        
]

chungbuk_food_24 = [{
    "predict_month": "2024년01월_food_chungbuk",
    "predict_value": chungbuk_food_24_jan
}, {
    "predict_month": "2024년02월_food_chungbuk",
    "predict_value": chungbuk_food_24_feb
},
 {
    "predict_month": "2024년03월_food_chungbuk",
    "predict_value": chungbuk_food_24_mar
},
              
   {
    "predict_month": "2024년04월_food_chungbuk",
    "predict_value": chungbuk_food_24_apr
},
{
    "predict_month": "2024년05월_food_chungbuk",
    "predict_value": chungbuk_food_24_may
},
 {
    "predict_month": "2024년06월_food_chungbuk",
    "predict_value": chungbuk_food_24_jun
},
 {
    "predict_month": "2024년07월_food_chungbuk",
    "predict_value": chungbuk_food_24_jul
},
{
    "predict_month": "2024년08월_food_chungbuk",
    "predict_value": chungbuk_food_24_aug
},
 {
    "predict_month": "2024년09월_food_chungbuk",
    "predict_value": chungbuk_food_24_sep
},

         {
    "predict_month": "2024년10월_food_chungbuk",
    "predict_value": chungbuk_food_24_oct
},
{
    "predict_month": "2024년11월_food_chungbuk",
    "predict_value": chungbuk_food_24_nov
},
 {
    "predict_month": "2024년12월_food_chungbuk",
    "predict_value": chungbuk_food_24_dec
},
        
]

In [13]:
conn=ora.connect('C##PROJECT/1@192.168.1.100:1521/xe')

In [14]:
cursor = conn.cursor()
print(cursor)

<oracledb.Cursor on <oracledb.Connection to C##PROJECT@192.168.1.100:1521/xe>>


In [15]:
for item in chungbuk_hotel_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]


    sql_insert = 'INSERT INTO chungbuk_hotel_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_condo_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_condo_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_camping_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_camping_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_elsesleep_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_elsesleep_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_dutyfree_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_dutyfree_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_leisureshop_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_leisureshop_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_hugeshop_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]


    sql_insert = 'INSERT INTO chungbuk_hugeshop_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_amuse_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_amuse_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_golf_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_golf_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_ski_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_ski_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_culture_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_culture_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()

for item in chungbuk_food_24:
    predict_month = item["predict_month"]
    predict_value = item["predict_value"]

    sql_insert = 'INSERT INTO chungbuk_food_24 (predict_month, predict_value) VALUES (:predict_month, :predict_value)'

    cursor.execute(sql_insert, predict_month=predict_month, predict_value=predict_value)

conn.commit()